# Como trabalhar com dados ruins

In [88]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [237]:
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Ex12") \
    .getOrCreate()

## Com o modo PERMISSIVE, os dados aparecem

In [238]:
spark.read.option('mode', 'PERMISSIVE').csv('./data/data.csv', header=True, inferSchema=True).show()

+-------------+-----+------+----+
|           Id| Nome|Estado|Nota|
+-------------+-----+------+----+
|            1|Pedro|    SP|  80|
|            2|Maria|    RJ| 100|
|            3|  Ana|    SP|  20|
|            4| Joao|    MG|  35|
|5|Lucas|SC|40| null|  null|null|
+-------------+-----+------+----+



## Com o modo DROPMALFORMED, os dados são desconsiderados

In [239]:
spark.read.option('mode', 'DROPMALFORMED').csv('./data/data.csv', header=True, inferSchema=True).show()

+---+-----+------+----+
| Id| Nome|Estado|Nota|
+---+-----+------+----+
|  1|Pedro|    SP|  80|
|  2|Maria|    RJ| 100|
|  3|  Ana|    SP|  20|
|  4| Joao|    MG|  35|
+---+-----+------+----+



## Com o modo FAILFAST, há uma exceção, levantando um erro

In [240]:
spark.read.option('mode', 'FAILFAST').csv('./data/data.csv', header=True, inferSchema=True).show()

22/07/05 21:06:23 ERROR Executor: Exception in task 0.0 in stage 102.0 (TID 101)
org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:395)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.B

Py4JJavaError: An error occurred while calling o789.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 102.0 failed 1 times, most recent failure: Lost task 0.0 in stage 102.0 (TID 101, 10.0.0.135, executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:395)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext

## Criação do schema com uma coluna para dados corrompidos e ler o arquivo com columnNameofCorruptRecord para colocar os dados corrompidos na coluna "dados_corrompidos"

In [241]:
schema1 = StructType([StructField('Id', IntegerType(), True),
                     StructField('Nome', StringType(), True),
                     StructField('Estado', StringType(), True),
                     StructField('Nota', IntegerType(), True),
                     StructField('_Dados_corrompidos', StringType())])

df = spark.read.schema(schema
        ).option('columnNameofCorruptRecord', '_Dados_corrompidos'
        ).option('mode', 'PERMISSIVE'
        ).csv('./data/data.csv', header=True, inferSchema=True)

In [242]:
df.show()

+-------------+-----+------+----+------------------+
|           Id| Nome|Estado|Nota|_Dados_corrompidos|
+-------------+-----+------+----+------------------+
|            1|Pedro|    SP|  80|              null|
|            2|Maria|    RJ| 100|              null|
|            3|  Ana|    SP|  20|              null|
|            4| Joao|    MG|  35|              null|
|5|Lucas|SC|40| null|  null|null|     5|Lucas|SC|40|
+-------------+-----+------+----+------------------+



In [243]:
df.cache()
df.filter('_Dados_corrompidos is not null').show()
#df.unpersist()

+-------------+----+------+----+------------------+
|           Id|Nome|Estado|Nota|_Dados_corrompidos|
+-------------+----+------+----+------------------+
|5|Lucas|SC|40|null|  null|null|     5|Lucas|SC|40|
+-------------+----+------+----+------------------+



22/07/05 21:06:24 WARN CacheManager: Asked to cache already cached data.


In [244]:
df = df.withColumn('_Dados_corrompidos_sep', split(col('_Dados_corrompidos'), '[|]')).drop('_Dados_corrompidos')

In [245]:
df.show()

+-------------+-----+------+----+----------------------+
|           Id| Nome|Estado|Nota|_Dados_corrompidos_sep|
+-------------+-----+------+----+----------------------+
|            1|Pedro|    SP|  80|                  null|
|            2|Maria|    RJ| 100|                  null|
|            3|  Ana|    SP|  20|                  null|
|            4| Joao|    MG|  35|                  null|
|5|Lucas|SC|40| null|  null|null|    [5, Lucas, SC, 40]|
+-------------+-----+------+----+----------------------+



In [246]:
df.filter('_Dados_corrompidos_sep is not null').show()

+-------------+----+------+----+----------------------+
|           Id|Nome|Estado|Nota|_Dados_corrompidos_sep|
+-------------+----+------+----+----------------------+
|5|Lucas|SC|40|null|  null|null|    [5, Lucas, SC, 40]|
+-------------+----+------+----+----------------------+



### Coleta dos dados como uma lista

In [247]:
linhas = df.collect()[4][4]
print(linhas)

['5', 'Lucas', 'SC', '40']


In [248]:
linhas[0] = eval(linhas[0])
linhas[3] = eval(linhas[3])

In [249]:
linhas = [tuple(linhas)]

In [250]:
type(linhas)

list

In [251]:
type(linhas[0][0])

int

In [252]:
print(linhas)

[(5, 'Lucas', 'SC', 40)]


### Criação do schema para a lista coletada

In [253]:
schema2 = StructType([StructField('Id', IntegerType(), True),
                     StructField('Nome', StringType(), True),
                     StructField('Estado', StringType(), True),
                     StructField('Nota', IntegerType(), True)])

### Criação de um segundo dataframe, deleção da coluna "_Dados_corrompidos_sep", para conseguir usar o join entre os dois df e a deleção da linha corrompida

In [254]:
df2 = spark.createDataFrame(data=linhas, schema=schema2)

In [255]:
df2.show()

+---+-----+------+----+
| Id| Nome|Estado|Nota|
+---+-----+------+----+
|  5|Lucas|    SC|  40|
+---+-----+------+----+



In [256]:
df = df.drop('_Dados_corrompidos_sep')

In [257]:
df.union(df2).dropna().show()

+---+-----+------+----+
| Id| Nome|Estado|Nota|
+---+-----+------+----+
|  1|Pedro|    SP|  80|
|  2|Maria|    RJ| 100|
|  3|  Ana|    SP|  20|
|  4| Joao|    MG|  35|
|  5|Lucas|    SC|  40|
+---+-----+------+----+

